In [1]:
import csv
import sys, os
import pandas as pd
import numpy as np

The basic code outline is as follows:   
0. Import both *talks_6e.csv* and *metadata_all.csv*. Create output file *talks_6f.csv*
1. Sort each import by the author name
2. ~~For *metadata_all.csv*, create a new colum that breaks the slug up into a list of lower-cased words.~~ 
3. Create an index list for *metadata_all.csv*. The idea of this list is to remove rows from consideration as we match the slugs to rows in *talks_6e.csv*.
4. Loop over each row in *talks_6e.csv*:
  1. Create two new lists to use: 
    * The list of names, all lower cased
    * The list of words in the title, all lowered case
  2. Iterate over each available row in *metadata_all.csv*, matching the author lists to the slug lists. 
    * If the author and slug lists intersect over at least 50% of the words in the author list, then compare the title and slug lists
    * If the title and slug lists match over 50% of the words in the title list, then match these rows and create a new row in the output that has the information from both *talks_6e.csv* and *metadata_all.csv*. Then move on to the next row of *metadata_all.csv*.
    * If no match is found, move on to the next row of *metadata_all.csv*.

In [7]:
# Step 0 - Import CSV files
read1 = csv.reader(open('../data/talks_6e.csv'))
headers = next(read1)

read2 = csv.reader(open('../../tedmetadata/metadata_all.csv'))
head2 = next(read2)

# Create headers for the output file, such that:
#    * 'date' --> 'published'
#    * 'numDate' --> numPub'

ind = headers.index('date')
headers[ind] = 'published'

ind = headers.index('numDate')
headers[ind] = 'numPub'

# 2) Add headers from the *metadata_all.csv* file
headers = [head2[0]] + headers[1:] + head2[1:] + ['numFilmed']

# Set index names:
#aund = headers.index('author')
#tnd = headers.index('title')



In [18]:
#test = next(read1)
#print(type(test))

In [19]:
#print(headers)
#print(head2)

In [20]:
#len(talks6)

In [21]:
#talks6.iloc[0].name

In [22]:
#talks6.loc[talks6.iloc[0].name].tolist()[1:]

In [23]:
#row_test = talks6.loc[talks6.iloc[0].name]
#row_test["author"].lower().split()

Here's the thinking.... maybe using pandas we can do all the things in place? Create a few fake lists that we add values to as we go. Then create a new pandas dataframe that has all the pieces. 

It seems to be easy to add a column to a pandas dataframe, but you have to have a full column. So basically, we pull each talk, add the info we need to new lists at the index that things will appear (using a hack of iloc+loc) and then add the new columns after the loop....

In [8]:
# Step 1 - Sort each import: one by slug, the other by author
read1pd = pd.read_csv('../data/talks_6e.csv')
talks6 = read1pd.sort_values("author", kind="mergesort")

# Import *metadata_all.csv* as pandas data frame
meta_raw = pd.read_csv('../../tedmetadata/metadata_all.csv')
meta = meta_raw.sort_values("slug", kind="mergesort")

In [25]:
#row2 = meta.iloc[2]
#slist = row2['slug']
#type(slist)

In [26]:
# Step 2 - For *metadata_all.csv*, create a dataframe with a new column where there is a list of words
# jk, we're just going to do this in the loop

In [9]:
# Step 3 - Create an index list for *metadata_all.csv*
mnds = list(range(len(meta)))

In [11]:
# Step 4 - Loop over the rows in *talks_6e.csv*
with open('../data/talks_7.csv', 'w') as csvfile:
    # Make the output file
    outfile = csv.writer(csvfile)
    outfile.writerow(headers)
    
    matchless_inds = []
    
    # Loop over each row in *talks_6e.csv*
    for inds in range(len(talks6)):
        talk_row = talks6.loc[talks6.iloc[inds].name]
        aulist = talk_row["author"].lower().split()
        tlist = talk_row["title"].lower().split()
        
        # Set MATCH flag
        match = False
        
        for m in mnds:
            # Grab row associated to index M in *metadata_all.csv*
            row2 = meta.iloc[m]
            slist = row2['slug'].lower().split('_')
            
            # Compare author and slug lists:
            if len(set(aulist) & set(slist)) >= 0.5*len(aulist):
                # If comparison goes well, then compare title and slug lists:
                if len(set(tlist) & set(slist)) > 0.5*len(tlist):
                    match = True
                    
            # If we find a match, then we need to add the output row:
            if match: 
                add_row = [row2['slug']] + talk_row.tolist()[1:]
                add_row.append(row2['view_count'])
                add_row.append(row2['event'])
                add_row.append(row2['filmed'])
                numFilmed = ''.join(row2["filmed"].split('-'))
                add_row.append(numFilmed)
                
                outfile.writerow(add_row)
                
                mnds.remove(m)
                break
        if match == False:
            matchless_inds.append(inds)

In [19]:
len(mnds)
len(meta)
len(talks6)

2067

In [16]:
# Output files test

unmatchedMeta = meta.iloc[mnds]
unmatchedTalks = talks6.iloc[matchless_inds]

,slug,view_count,event,filmed
0,9_11_healing_the_mothers_who_found_forgiveness...,836750,TEDWomen 2010,2010-12-12
1,a_choir_as_big_as_the_internet,400751,Eric Whitacre's Virtual Choir,2010-03-10
3,a_robot_that_flies_like_a_bird,6331411,TEDGlobal 2011,2011-07-15
4,a_ted_speaker_s_worst_nightmare,2984064,TED2012,2012-03-01


In [33]:
test = [2,4,5,6]
for m in test:
    print(m)
    print(test)
    test.remove(m)
    print(test)

2
[2, 4, 5, 6]
[4, 5, 6]
5
[4, 5, 6]
[4, 6]


In [35]:
out = pd.read_csv('../data/talks_7.csv')
len(out)

1375